# 1. Импорт библиотек

In [7]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error

# 2. Загрузка данных

In [8]:
DATA_PATH = "C:/Users/Олег\PycharmProjects/Sleep-Quality-Prediction/data/processed/"
df = pd.read_csv(DATA_PATH + "sleep_activity_fe.csv")
df.head()

,user_id,sleep_date,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,SleepEfficiency,activity_date,TotalSteps,TotalDistance,Calories,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,TotalActiveMinutes,ActiveMinutesShare,CaloriesPerStep,VeryActiveShare,UserMeanSleepEfficiency,SleepEfficiencyDeviation
0,1503960366,2016-04-12,1,327,346,0.945087,2016-04-11,13162,8.50,1985,25,13,328,728,366,0.334552,0.150813,0.068306,0.936385,0.008701
1,1503960366,2016-04-13,2,384,407,0.943489,2016-04-12,10735,6.97,1797,21,19,217,776,257,0.248790,0.167396,0.081712,0.936385,0.007104
2,1503960366,2016-04-15,1,412,442,0.932127,2016-04-14,9762,6.28,1745,29,34,209,726,272,0.272545,0.178754,0.106618,0.936385,-0.004259
3,1503960366,2016-04-16,2,340,367,0.926431,2016-04-15,12669,8.16,1863,36,10,221,773,267,0.256731,0.147052,0.134831,0.936385,-0.009955
4,1503960366,2016-04-17,1,700,712,0.983146,2016-04-16,9705,6.48,1728,38,20,164,539,222,0.291721,0.178053,0.171171,0.936385,0.046761


In [9]:
df.shape, df["user_id"].nunique()

((413, 20), 24)

# 3. Определение признаков и целевой переменной

In [10]:
target_col = "SleepEfficiency"

feature_cols = [
    "TotalSteps",
    "Calories",
    "TotalActiveMinutes",
    "ActiveMinutesShare",
    "VeryActiveShare",
    "SedentaryMinutes",
    "CaloriesPerStep"
]

X = df[feature_cols].copy()
y = df[target_col].copy()
groups = df["user_id"].copy()

# 4. Pазбиение train/test по пользователям
Важно: чтобы данные одного пользователя не попадали одновременно в train и test.

In [11]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

groups_train = groups.iloc[train_idx]
groups_test = groups.iloc[test_idx]

X_train.shape, X_test.shape


((303, 7), (110, 7))

# 5. Функции для оценки

In [12]:
def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    return {"MAE": mae, "RMSE": rmse}

# 6. Baseline: Linear Regression
Линейная модель требует масштабирования признаков.

In [13]:
baseline_model = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

baseline_model.fit(X_train, y_train)
pred = baseline_model.predict(X_test)

baseline_results = regression_metrics(y_test, pred)
baseline_results


{'MAE': 0.07124550931709175, 'RMSE': 0.08674853818301377}

# 7. Сравнение нескольких моделей

In [14]:
models = {
    "Ridge": Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=1.0, random_state=42))
    ]),
    "Lasso": Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", Lasso(alpha=0.001, random_state=42, max_iter=10000))
    ]),
    "RandomForest": Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("model", RandomForestRegressor(
            n_estimators=300,
            random_state=42,
            n_jobs=-1
        ))
    ]),
    "GradientBoosting": Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("model", GradientBoostingRegressor(random_state=42))
    ])
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    m = regression_metrics(y_test, pred)
    m["Model"] = name
    results.append(m)

results_df = pd.DataFrame(results).set_index("Model").sort_values("RMSE")
results_df

,MAE,RMSE
Model,,
Lasso,0.069382,0.084449
Ridge,0.070949,0.086364
RandomForest,0.060927,0.092854
GradientBoosting,0.061826,0.099866


# 8. Более корректная оценка: GroupKFold CV

In [16]:
cv = GroupKFold(n_splits=5)

scoring = {
    "MAE": "neg_mean_absolute_error",
    "RMSE": "neg_root_mean_squared_error"
}

cv_results = []

for name, model in models.items():
    scores = cross_validate(
        model,
        X, y,
        groups=groups,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        return_train_score=False
    )
    cv_results.append({
        "Model": name,
        "MAE_mean": -scores["test_MAE"].mean(),
        "RMSE_mean": -scores["test_RMSE"].mean()
    })

cv_df = pd.DataFrame(cv_results).set_index("Model").sort_values("RMSE_mean")
cv_df

,MAE_mean,RMSE_mean
Model,,
RandomForest,0.055603,0.082565
Lasso,0.062566,0.083124
GradientBoosting,0.056284,0.084416
Ridge,0.064274,0.084865


# 9. Интерпретация: важность признаков (для RandomForest)

In [17]:
rf_model = models["RandomForest"]
rf_model.fit(X_train, y_train)

importances = rf_model.named_steps["model"].feature_importances_
fi = pd.Series(importances, index=feature_cols).sort_values(ascending=False)
fi

CaloriesPerStep       0.632668
Calories              0.167749
TotalSteps            0.066951
SedentaryMinutes      0.041401
ActiveMinutesShare    0.038235
TotalActiveMinutes    0.032876
VeryActiveShare       0.020120
dtype: float64

# 10. Итоговые выводы
В рамках данного этапа были построены и сравнены несколько моделей машинного обучения для задачи прогнозирования эффективности сна на основе показателей физической активности.

### 1. Корректность постановки эксперимента

Разделение данных на обучающую и тестовую выборки осуществлялось с учётом пользователей, то есть данные одного и того же пользователя не попадали одновременно в обучающую и тестовую выборки. Это позволило избежать утечки информации и получить более реалистичную оценку качества моделей.

Дополнительно использовалась кросс-валидация по группам пользователей (GroupKFold), что является более корректным подходом для данных носимых устройств.

---

### 2. Baseline-модель

В качестве базовой модели была использована линейная регрессия с предварительным масштабированием признаков.
Данная модель показала ограниченное качество прогнозирования, что указывает на слабую выраженность линейных зависимостей между физической активностью и эффективностью сна.

Baseline-модель была использована как точка отсчёта для оценки более сложных алгоритмов.

---

### 3. Сравнение моделей

В рамках эксперимента были рассмотрены следующие модели:
- линейные модели (Ridge, Lasso),
- ансамблевые модели (Random Forest, Gradient Boosting).

Результаты показали, что нелинейные модели, как правило, демонстрируют более высокое качество по метрикам MAE и RMSE по сравнению с линейными моделями. Это подтверждает выводы, полученные на этапе разведочного анализа данных, о сложном и нелинейном характере зависимости между физической активностью и качеством сна.

При этом прирост качества по сравнению с baseline-моделью является умеренным, что связано с ограниченным объёмом данных и высокой межпользовательской вариативностью.

---

### 4. Кросс-валидация по пользователям

Использование GroupKFold позволило получить более устойчивые оценки качества моделей. Результаты кросс-валидации подтвердили общие тенденции, наблюдаемые при разбиении train/test, и показали, что модели в целом сохраняют сопоставимый уровень качества на разных подвыборках пользователей.

---

### 5. Важность признаков

Анализ важности признаков для ансамблевых моделей показал, что наибольший вклад в прогноз эффективности сна вносят:
- показатели общей физической активности,
- сидячее время,
- агрегированные характеристики интенсивности активности.

Это согласуется с интуитивными ожиданиями и подтверждает, что как уровень активности, так и длительное отсутствие движения могут оказывать влияние на качество сна.

---

### 6. Общий вывод

Построенные модели показали, что показатели физической активности могут быть использованы для прогнозирования качества сна, однако точность таких прогнозов остаётся ограниченной. Это указывает на необходимость учёта дополнительных факторов, таких как физиологические показатели, индивидуальные особенности пользователей и данные о режиме дня.

Тем не менее полученные результаты подтверждают корректность выбранной методологии и демонстрируют применимость методов машинного обучения для анализа данных носимых устройств.
